In [8]:
import pandas as pd
import sklearn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

In [9]:
# read in dataset and select columns
terror = pd.read_csv('/home/hzsit/Documents/CZ4032/CZ4032-Data-Analytics-and-Mining/terrorism_new.csv')

cols = ["iyear", "extended", "latitude", "longitude", "attacktype1"
        , "success", "suicide", "targtype1", "nperps", "gname",
        "nkill", "nwound", "property"]

terror = terror[cols]

def removeUnknown(df_in, col, match):
    df = df_in.copy()
    df = df.loc[df[col] != match]
    return df

#remove all unknown
for col in cols:
    terror = removeUnknown(terror, col, 'Unknown')
    terror = removeUnknown(terror, col, -9)
    terror = removeUnknown(terror, col, -99)
    
terror = terror.dropna()
terror

/home/hzsit/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,iyear,extended,latitude,longitude,attacktype1,success,suicide,targtype1,nperps,gname,nkill,nwound,property
1,1970,0,19.371887,-99.086624,6,1,0,7,7.0,23rd of September Communist League,0.0,0.0,0
6,1970,0,-34.891151,-56.187214,1,0,0,3,3.0,Tupamaros (Uruguay),0.0,0.0,0
8,1970,0,43.076592,-89.412488,7,1,0,4,1.0,New Year's Gang,0.0,0.0,1
9,1970,0,43.072950,-89.386694,7,1,0,2,1.0,New Year's Gang,0.0,0.0,1
21,1970,0,-34.891151,-56.187214,2,1,0,8,5.0,Tupamaros (Uruguay),0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191430,2018,0,-1.119683,120.447285,6,1,0,1,12.0,Mujahidin Indonesia Timur (MIT),1.0,0.0,0
191445,2018,0,53.487144,-2.242375,2,1,0,19,1.0,Jihadi-inspired extremists,0.0,3.0,0
191451,2018,0,5.963716,10.159570,1,0,0,2,20.0,Separatists,0.0,1.0,0
191453,2018,0,-1.119683,120.447285,2,1,0,3,12.0,Mujahidin Indonesia Timur (MIT),0.0,2.0,0


In [10]:
le = preprocessing.LabelEncoder()
gname_cat = le.fit_transform(terror['gname'])
terror['gname'] = gname_cat


In [11]:
label = 'gname'
classifier = RandomForestClassifier(n_estimators = 100)

In [12]:
def validateRFE(df, classifier, num_features, label):
    x, y = terror.drop([label],axis=1), terror[label]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    classifier.fit(x_train, y_train)
    pre_rfe_score = classifier.score(x_test,y_test)
    print(f"Pre-RFE: {pre_rfe_score} [label : {label}]")
    
    rfe = RFE(estimator=classifier, n_features_to_select=num_features)
    rfe.fit(x_train, y_train)
    x_train_rfe = rfe.transform(x_train)
    x_test_rfe = rfe.transform(x_test)
    classifier.fit(x_train_rfe,y_train)
    post_rfe_score = classifier.score(x_test_rfe, y_test)
    print(f"Post-RFE [no. of features: {num_features}] [label : {label}]: {post_rfe_score}")

In [ ]:
validateRFE(terror, classifier, 3, label)

Pre-RFE: 0.6741816605246044 [label : gname]
